# Label extraction

### Testing BART c

In [ ]:
from transformers import pipeline
# Define the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [ ]:
# Text to classify
text = "This video explains the basics of Python programming and coding."

# Define possible categories
candidate_labels = [
    "tutorial",
    "lecture",
    "explainer",
    "study tips",
    "experiment",
    "interview",
    "language learning",
    "professional development"
]

# Perform zero-shot classification
result = classifier(text, candidate_labels)

# Output the results
print(result)